In [1]:
import sys, os
from omegaconf import OmegaConf
import torch
import numpy as np
import pandas as pd

sys.path.append("/home/jgershon/git/cleo")
from policy_utils import PolicyMPNN, alphabet


test_pdb = "/home/jgershon/projects/itopt/declan/TS1_trp_6_conformers_0008_000_10-atomized-bb-False_4_9_MPNN.pdb"

config = {
    "run_name": "test_v1",
    "output_dir": "/home/jgershon/projects/itopt/policy_mpnn/testing",
    "model_type": "ligand_mpnn",
    "N_train": 500,
    "pdb": test_pdb,
    "batch_size": 16,
    "temperature": 1.0,
    "omit_AA": "CX",
    "lr": 1e-4,
    "fixed_residues": "A29 A30 A31 A32 A60 A61 A62 A63 A100 A101 A102 A103",
    "checkpoint_every_n_steps": 10,
    "reward": {
        "_target_": "reward_utils.EnrichAminoAcidReward",
        "AA_to_enrich": "E"
    },
    "eval":False,
}

config = OmegaConf.create(config)

policy_mpnn = PolicyMPNN(config)


/home/jgershon/git/cleo/policy_utils.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=self.device)


In [4]:
# to train policy
policy_mpnn.train()

Training:   8%|▊         | 39/500 [00:33<06:34,  1.17it/s]


KeyboardInterrupt: 

In [5]:
path = "/home/jgershon/projects/itopt/policy_mpnn/testing/test_v1_train_metrics.csv"
df = pd.read_csv(path)

In [14]:
sum([x.flatten().shape[0] for x in policy_mpnn.model.parameters() if x.requires_grad])

1155093